In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../src')
sys.path.append('../../msq-domain/src')
sys.path.append('../../quant_container/src')

import aiostreams.operators as op
from aiostreams.config import QCConfigProvider
# this is where AsyncKafkaSource, AsyncKafkaPublisher get the Kafka broker location
QCConfigProvider().kafka_broker = 'kafka:9092'

print(QCConfigProvider().kafka_broker)

kafka:9092


In [2]:
# try writing just one record to Kafka using the basic kafka library, 
# and reading back from it, to make sure Kafka is up, 
from kafka import KafkaProducer, KafkaConsumer
import pickle

topic='test_topic_2'
kafka_host = QCConfigProvider().kafka_broker

kprod=KafkaProducer(bootstrap_servers=kafka_host)
kprod.send(topic,pickle.dumps('test'))

consumer = KafkaConsumer(topic,bootstrap_servers=kafka_host,auto_offset_reset='earliest',
                         group_id=None, value_deserializer=lambda x: pickle.loads(x))
next(consumer).value

'test'

In [3]:
# let's check QC is actually getting data
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('bond-trades-topic', \
                               value_deserializer =  lambda x: x.decode('utf-8'))| \
                                op.map(lambda x: json_to_domain(x.value, historical = True) )>[]#
                               #op.map(lambda x: print(x.value) > print# #op.map(lambda x: json.loads(x.value) ) > []                            
    # #op.map(lambda x: json.loads(x.value) ) > []
                           #
                            #
                #
                
run(input_graph)
print(input_graph.sink[0])

# json_to_domain(input_graph.sink[0].value)

(<mosaicsmartdata.core.trade.FixedIncomeBondTrade object at 0x000002244DDCE898>, [<mosaicsmartdata.core.quote.Quote object at 0x000002244DE54D30>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DE54DD8>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DDCE160>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DDCE940>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DDCE7F0>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DDD7F98>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DDD7748>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DDF3400>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DE3D978>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DE3D9B0>, <mosaicsmartdata.core.quote.Quote object at 0x000002244DE3D9E8>])


In [4]:
# Look at what QC is outputting inside docker-compose
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('output-topic',\
                               value_deserializer =  lambda x: x.decode('utf-8'))|\
                op.map(lambda x: x.value ) > print
                #op.map(lambda x: json_to_domain(x.value)) > print
                #op.map(lambda x: json.loads(x.value) ) > print
                
run(input_graph)

{"negotiationId": "2", "legMarkout": [{"markoutPeriod": {"timeUnit": "SECONDS", "value": "-900"}, "evaluatedPricingSource": "INTERNAL", "markoutPrice": [{"value": -840.2320000000003, "priceType": "UNHEDGED_SPREAD_PRICE"}, {"value": -94.08786343020628, "priceType": "UNHEDGED_INITIAL_EDGE"}]}], "instrumentId": "912796KN8", "cashflowDirection": "+", "dateLocalDate": "2017-09-21", "venueName": "TRADEWEBUST_ASIA"}
{"negotiationId": "2", "legMarkout": [{"markoutPeriod": {"timeUnit": "SECONDS", "value": "-60"}, "evaluatedPricingSource": "INTERNAL", "markoutPrice": [{"value": -841.7950000000005, "priceType": "UNHEDGED_SPREAD_PRICE"}, {"value": -94.26288572231302, "priceType": "UNHEDGED_INITIAL_EDGE"}]}], "instrumentId": "912796KN8", "cashflowDirection": "+", "dateLocalDate": "2017-09-21", "venueName": "TRADEWEBUST_ASIA"}
{"negotiationId": "2", "legMarkout": [{"markoutPeriod": {"timeUnit": "SECONDS", "value": "0"}, "evaluatedPricingSource": "INTERNAL", "markoutPrice": [{"value": -837.1070000000

In [5]:
# Let's load a snapshot of the graph locally and run against the same data
from aiostreams import AsyncKafkaSource, AsyncKafkaPublisher, run
import aiostreams.operators as op
from aiostreams.persist import KafkaPersister
from mosaicsmartdata.common.json_convertor import *
import json

graph_name = 'test_pid__markouts_unhedged' # got that name from the logs
class GetFirst:
    def __init__(self):
        self.count = -1

    def __call__(self, x):
        self.count += 1
        if not self.count:
            return True
        else:
            return False
        
p = KafkaPersister()
reflated_graph = p.load(graph_name, GetFirst())
reflated_graph.persistence_policy = None # don't want to persist states when we re-run
reflated_graph.sink = [] #so we don't pollute the original output topic
print(reflated_graph.message_count) 
# as the graph is set to run 'forever' you want to manually interrupt it after a while
run(reflated_graph)


1


KeyboardInterrupt: 

In [6]:
print(len(reflated_graph.sink))

59
